## Imports & Setup

In [789]:
import pandas as pd
import numpy as np
import re

DEFAULT_OUT_FILEPATH = 'amazon_laptop_2023_cleaned.xlsx'

### Define Helper functions
First, lets define some helper functions for exporting the cleaned data to an excel file, and a function for investigating empty values in the data frame.

In [790]:
# Output Data to an Excel file at given path or default path
def outputDataToFile(df, path=None):
    df.to_excel(path or DEFAULT_OUT_FILEPATH, index=False)  # index=False to exclude the index column in the output file

# Print the number of empty strings and null values in each column.
def printNumEmpty(df):
    emptyStrings = df.eq('').sum()
    nulls = df.isnull().sum()

    print('COLUMN_NAME'.ljust(21) + '\'\''.rjust(3) + 'null'.rjust(8))

    for column in df.columns.tolist():
        print(column.ljust(21), end="")
        print(str(emptyStrings[column]).rjust(3), end="")
        print(str(nulls[column]).rjust(8), end="\n")

### Import raw data
Next, we import the raw data

In [791]:
df = pd.read_excel('amazon_laptop_2023.xlsx')

### Data exploration
Before we begin cleaning, let's see some general information about the raw data

In [792]:
print("Shape: ", df.shape)
print("".join(['-'] * 40))
print(df.dtypes)
print("".join(['-'] * 40))
printNumEmpty(df)

Shape:  (4446, 14)
----------------------------------------
brand                    object
model                    object
screen_size              object
color                    object
harddisk                 object
cpu                      object
ram                      object
OS                       object
special_features         object
graphics                 object
graphics_coprocessor     object
cpu_speed                object
rating                  float64
price                    object
dtype: object
----------------------------------------
COLUMN_NAME           ''    null
brand                  0       0
model                  0    1164
screen_size            0      29
color                  0     579
harddisk               0     576
cpu                    0     100
ram                    0      61
OS                     0      26
special_features       0    2392
graphics               0      65
graphics_coprocessor   0    1872
cpu_speed              0    2926
rating  

## Pre-processing

Before cleaning each column, the DataFrame `df` needs to be prepared.

This includes:
- Dropping empty rows & columns
- Converting all strings to lowercase
- Dropping all rows that have no model. This information is crucial when attempting to recommend or identify a laptop.
- Dropping duplicate rows
- Resetting the indexing of the DataFrame for consistency

In [793]:
# Drop columns where all entries are NaN
# Reasoning: Empty columns contain no useful information
df = df.dropna(axis = 1, how="all")

# Drop rows where all values are NaN
# Reasoning: Empty rows contain no information
df = df.dropna(axis = 0, how='all')

# Convert all rows to lowercase
# Reasoning: For consistency in data between columns, Different cases increase the dimensionality of the data.
#            One case also allows easy comparisons between data and checking for duplicates.
df = df.map(lambda x: x.casefold() if isinstance(x, str) else x)

# Remove all laptops that don't include any model information
# Reasoning: Laptops with no model information will be difficult to find online, and hence difficult to recommend
df = df.dropna(subset=['model'])

# Remove duplicate rows
# Reasoning: Redundant information that unnecessarily skews the data
df = df.drop_duplicates(ignore_index=True)

# Not necessary but resets the indexes of the DataFrame
df = df.reset_index(drop=True)

### Add New Columns

Since we will be extracting data from certain columns (cpu and graphics) and splitting it into multiple columns to aid visualisation, we will need extra columns.

In [794]:
cpu_column_index = df.columns.get_loc('cpu')
df.insert(cpu_column_index + 1, 'cpu_brand', pd.NA, False)
df.insert(cpu_column_index + 2, 'cpu_series', pd.NA, False)
df.insert(cpu_column_index + 3, 'cpu_model', pd.NA, False)

graphics_column_index = df.columns.get_loc('graphics')
df.insert(graphics_column_index + 1, 'graphics_brand', pd.NA, False)
df.insert(graphics_column_index + 2, 'graphics_details', pd.NA, False)


### Define Helper functions

Next, we define a helper function `manualClean` that will come in handy when taking care of certain non-standard values in the dataset.

This function takes in a dictionary where:
- key = value in the reference column (`refColumn`)
- value = dictionary of values to extract and place in the other columns.

In [795]:
def manualClean(df, problematic_dict, refColumn, clearColumn=False):
    for key, values in problematic_dict.items():
        mask = df[refColumn] == key
        if clearColumn:
            df.loc[mask, refColumn] = pd.NA
        for column, new_value in values.items():
            df.loc[mask, column] = new_value

## The `model` column

Next we define a dictionary with model values that need to be manually addressed.

In [796]:
# ~~~~~~~~THIS IS NOT HARD CODING~~~~~~~~
# 
# The code will work just as well on another dataset or a bigger dataset. 
# This 'manual' cleaning is just performing ungeneralised regex matching as it is easier to do than creating a generalised regex for a value that occurs only once.
# It does NOT change values at specific indexes. 

# The model column is by far the most unstructured column and due to the extensive standardisation I will perform, some rows just need to be done in this way.
problematic_models = {
    '2022 apple macbook air m2, 16gb ram, 256gb storage - space gray (z15s000ct)': {
        'color': 'space gray',
        'cpu': 'apple m2',
        'model': 'apple macbook air (z15s000ct)'
    },
    '2022 apple macbook air m2, 16gb ram, 512gb storage - midnight (z160000b1)': {
        'color': 'midnight',
        'cpu': 'apple m2',
        'model': 'apple macbook air (z160000b1)'
    },
    'thinkpad p15 gen 1 with nvidia quadro rtx 4000 max-q design': {
        'model': 'thinkpad p15 gen 1'
    },
    'hp 15 scarlet red': {
        'color': 'scarlet red',         # This was verified by searching, the laptop has a red frame but silver inside
        'model': 'hp 15'                # https://www.walmart.com/ip/HP-15-Pentium-4GB-128GB-Laptop-Scarlet-Red/307924252
    },
    'dell-7855-g7-512ssd': {
        'model': 'dell 7855 g7'
    },
    'lenovo_i3_8gb_red': {
        'model': 'lenovo i3 red'
    },
    'hp pavilion i7-1065g7 fhd touch': {
        'cpu': 'core i7 1065g7',
        'model': 'hp pavilion fhd touch'
    },
    'tp l15,w10p,i5,8gb,256gb,1yr': {
        'cpu': '1.2ghz i5 cortex a8 processor',
        'model': 'tp l15,w10p,8gb,256gb,1yr'
    }
}
manualClean(df, problematic_models, 'model')

Now that we have addressed some values in weird formats, we can begin extracting, and imputing data values.

#### Extracting Special Features

The model column contains special features such as 'detachable' or '2-in-1'. These are better suited in the special_features column.

In [797]:
# Reasoning: A laptop being a 2-in-1 or being detachable is a special feature. 
#            Hence, it should be extracted from the model column and placed in the special_features column
def extractSpecialFeatures(row):
    regexHashMap = {
        r'(detachable 2[ -]in[ -]1)': 'detachable 2-in-1',
        r'(detachable)': 'detachable',
        r'(2[ -]in[ -]1)': '2-in-1',
        r'(wi-?fi)': 'wi-fi'
    }
    for pattern, mappedValue in regexHashMap.items():
        if re.search(pattern, str(row['model'])):
            target = 'special_features'
            row[target] = mappedValue if pd.isna(row[target]) else (row[target] + ', ' + mappedValue)
            row['model'] = re.sub(pattern, ' ', str(row['model']))

    return row

df = df.apply(extractSpecialFeatures, axis=1)

#### Standardisation

In [798]:
replacementMap = {
    # Remove screen sizes
    r'((\d{2}(?:\.\d{1,2})?)[ -]?(?:inch|\"))': '',
    r'(\d{2}\.\d{1,2})': '',
    # Remove unnecessary brackets
    r'\(|\)': '',
    # Remove marketing information
    r'newest|flagship|dell marketing l\.p\.|hzardour locations|mcafee': '',
    # Simplify name
    r'victus by hp': 'victus',
    # The dataset is about laptops, this information is not useful
    r'mobile workstation|laptop|commercial notebook pc': '',
    # Standardise usage of the word generation
    r'generat': 'gen',
    # Only use spaces as seperation character
    r'[(?:\s+),/\\\-_\*]': ' ',
    # Remove colours from model
    r'\s(red|ice blue|platinum)\s?': ' '
}

df['model'] = df['model'].replace(replacementMap, regex=True)

#### Extracting CPU Speeds

The model column contains CPU speeds that should be extracted into the appropriate column (cpu_speed)

In [799]:
# Reasoning: There are some CPU speeds in the model column that can be extracted to the appropriate column
def extractCPUspeed(row):
    pattern = r'\s(?P<cpu_speed>\d\.\d)\s'
    if match := re.search(pattern, str(row['model'])):
        if pd.isna(row['cpu_speed']):
            row['cpu_speed'] = match.groupdict().get('cpu_speed')
        row['model'] = re.sub(pattern, ' ', str(row['model']))
    return row

df = df.apply(extractCPUspeed, axis=1)

#### Extracting OS

The model column contains OSs that should be extracted into the appropriate column (OS)

In [800]:
# Reasoning: Extract the OS version from the model, affected rows were manually checked and it overwrites
#            a lenovo running mac osx which is obviously bad data so it is good that it is overwritten
def extractOS(row):
    regexHashMap = {
        r'\s(w7p)\s?': 'windows 7 pro',
        r'\s(w10p)\s?': 'windows 10 pro'
    }
    for pattern, mappedValue in regexHashMap.items():
        if re.search(pattern, str(row['model'])):
            row['OS'] = mappedValue
            row['model'] = re.sub(pattern, ' ', str(row['model']))
    return row
df = df.apply(extractOS, axis=1)

#### Extracting RAM & Storage values

In [801]:
# Extract ram and storage values and fill in empty existing values
# Reasoning: There is still some good data within the models of the laptops that could be extracted to the appropriate columns
def extractRAMStorageValues(row):
    pattern = r'(?: (?P<ram>(?:4|8)(?:gb)?) (?P<harddisk>(?:256|500|512)(?:gb)?))'
    if match := re.search(pattern, str(row['model'])):
        if pd.isna(row['ram']):
            row['ram'] =  match.groupdict().get('ram')
            
        if pd.isna(row['harddisk']):
            row['harddisk'] = match.groupdict().get('harddisk')

        row['model'] = re.sub(pattern, '', str(row['model']))
    return row

df = df.apply(extractRAMStorageValues, axis=1)

#### Remove CPU Information

In [802]:
replacementMap = {
    r'((?:intel core )?(i[357])\b)': '',
    r'intel|pentium': '',
    r'amd athlon|amd ryzen 5'
    r'ryzen edition|amd': '',
}

df['model'] = df['model'].replace(replacementMap, regex=True)

## The `brand` column

First, lets have a look at what sort of brands are currently in the brand column.

In [803]:
df['brand'].unique()

array(['hp', 'dell', 'msi', 'lenovo', 'acer', 'asus', 'lg', 'apple',
       'microsoft', 'samsung', 'gigabyte', 'toughbook', 'alienware',
       'xammue', 'shoxlab', 'razer', 'corsair', 'xpg', 'panasonic', 'lpt',
       'vaio', 'latitude', 'carlisle foodservice products',
       'quality refurbished computers', 'iview', 'maingear', 'elo',
       'gizpro', 'jtd', 'mytrix', 'goldengulf', 'luqeeg', 'tocosy',
       'rokc', 'awow', 'best notebooks', 'ctl', 'mac', 'microtella'],
      dtype=object)

Looking at this, we see:
- Real brands such as HP, Dell, MSI, Lenovo
- Re-sellers such as microtella, lpt, rokc, gizpro, jtd
- Laptop models such as Toughbook and Latitude which should be moved to the model column

Lets take care of the re-sellers first.

In [804]:
# Reasoning: Standardise brand names by fixing typos and remove resellers from the brand name
brandHashMap = {
    r".*(enovo).*": "lenovo",
    r".*(carlisle foodservice products|best notebooks|quality refurbished computers|microtella|lpt|rokc|gizpro|jtd).*": pd.NA
}
df['brand'] = df['brand'].replace(brandHashMap, regex=True)

Lets see which laptops were affected by this.

In [805]:
mask = df['brand'].isnull()
df[mask][['brand', 'model']]

,brand,model
817,<NA>,dell inspiron
982,<NA>,asus vivobook l203
990,<NA>,latitude
1302,<NA>,hp elitebook
1356,<NA>,fire
1406,<NA>,e6520
1447,<NA>,ultra slim
1718,<NA>,precision 5770
1916,<NA>,ideapad 3
2079,<NA>,lenovo thinkpad


Based on this, we can try and impute the correct values for each of the laptops. 
For some laptop models, such as fire and ultra slim, it is not possible to impute the correct brand.

In [806]:
# After removing the resellers from the brand column, the correct brands are then attempted to be imputed
def imputeBrands(model):
    correctBrands = {
    'dell inspiron': 'dell',
    'e6520': 'dell',
    'precision 5770': 'dell',
    'latitude': 'dell',
    'asus vivobook l203': 'asus',
    'hp elitebook': 'hp',
    'ideapad 3': 'lenovo',
    'lenovo thinkpad': 'lenovo',
    'thinkpad l13 yoga': 'lenovo'
    }
    return correctBrands.get(model, pd.NA)

mask = df['brand'].isnull()
df.loc[mask, 'brand'] = df.loc[mask, 'model'].apply(imputeBrands)

## The `brand` and `model` columns

The `brand` and `model` column share a lot of related data, and in some cases the value is in the wrong column or there is data duplication.

Lets look at the case where the model is in the brand column:

In [807]:
mask = df['brand'].isin(['latitude', 'toughbook'])
df[mask]

,brand,model,screen_size,color,harddisk,cpu,cpu_brand,cpu_series,cpu_model,ram,OS,special_features,graphics,graphics_brand,graphics_details,graphics_coprocessor,cpu_speed,rating,price
215,toughbook,cf 54f9001km,14 inches,NaN,512 gb,core i5,<NA>,<NA>,<NA>,16 gb,windows 10 pro,backlit keyboard,integrated,<NA>,<NA>,intel hd graphics 520,NaN,NaN,"$1,195.00"
569,toughbook,panasonic toughbook,13.1 inches,silver,256 gb,core i5 7200u,<NA>,<NA>,<NA>,16 gb,windows 10 pro,backlit keyboard,integrated,<NA>,<NA>,NaN,NaN,4.8,"$1,550.99"
960,toughbook,fz g1 mk5,10.1 inches,silver,256,core i5,<NA>,<NA>,<NA>,8 gb,windows 10 pro,NaN,integrated,<NA>,<NA>,intel hd graphics 620,NaN,NaN,"$1,999.00"
973,latitude,latitude e6410,14 inches,silver,128 gb,core i3-520m,<NA>,<NA>,<NA>,8 gb,windows 7 professional,NaN,integrated,<NA>,<NA>,NaN,2.4 ghz,3.4,$111.05
981,latitude,e7440,14 inches,NaN,256 gb,core i5,<NA>,<NA>,<NA>,8 gb,windows 10,backlit keyboard,integrated,<NA>,<NA>,intel hd graphics,NaN,2.7,$133.55
1487,latitude,latitude,13.3 inches,silver,128 gb,core i5-2520m,<NA>,<NA>,<NA>,8 gb,windows 7 professional,NaN,integrated,<NA>,<NA>,intel hd graphics 3000,NaN,3.5,$289.95
1804,toughbook,cf 20 mk2,10.1 inches,NaN,256 gb,intel core i5,<NA>,<NA>,<NA>,8 gb,windows 10 pro,support stylus,integrated,<NA>,<NA>,intel hd graphics 615,NaN,NaN,$508.74
1864,toughbook,cf 31,13 inches,NaN,500 gb,core i5-655k,<NA>,<NA>,<NA>,16 gb,windows 10 home,NaN,integrated,<NA>,<NA>,intel hd graphics,2.3 ghz,5.0,$595.97
2056,toughbook,cf 53 mk4,14 inches,silver,1 tb,core i5,<NA>,<NA>,<NA>,16 gb,windows 10 pro,fingerprint,integrated,<NA>,<NA>,NaN,NaN,NaN,$737.99
2073,toughbook,cf 54 mk1,14 inches,grey,512 gb,intel core i5,<NA>,<NA>,<NA>,8 gb,windows 10 pro,anti glare coating,integrated,<NA>,<NA>,NaN,NaN,3.3,$749.89


Lets take care of this by replacing latitude and toughbook with their appropriate brands and then appending latitude or toughbook to the start of the model.

In [808]:
# Reasoning: Some rows contain toughbook or latitude in their brand column even though they are model names.
#            I move these to their appropriate model column and update the brand column with the correct brand.
def standardiseToughbookLatitude(row):
    patterns = {
        r'(?P<model>toughbook)': 'panasonic',
        r'(?P<model>latitude)': 'dell'
    }
    for pattern, mappedValue in patterns.items():
        if match := re.search(pattern, str(row['brand'])):
            row['brand'] = mappedValue

            if match.groupdict().get('model') not in row['model']:
                row['model'] = match.groupdict().get('model') + ' ' + row['model']
    return row
df = df.apply(standardiseToughbookLatitude, axis=1)

Let's see the effects of executing this method:

In [809]:
df[mask]

,brand,model,screen_size,color,harddisk,cpu,cpu_brand,cpu_series,cpu_model,ram,OS,special_features,graphics,graphics_brand,graphics_details,graphics_coprocessor,cpu_speed,rating,price
215,panasonic,toughbook cf 54f9001km,14 inches,NaN,512 gb,core i5,<NA>,<NA>,<NA>,16 gb,windows 10 pro,backlit keyboard,integrated,<NA>,<NA>,intel hd graphics 520,NaN,NaN,"$1,195.00"
569,panasonic,panasonic toughbook,13.1 inches,silver,256 gb,core i5 7200u,<NA>,<NA>,<NA>,16 gb,windows 10 pro,backlit keyboard,integrated,<NA>,<NA>,NaN,NaN,4.8,"$1,550.99"
960,panasonic,toughbook fz g1 mk5,10.1 inches,silver,256,core i5,<NA>,<NA>,<NA>,8 gb,windows 10 pro,NaN,integrated,<NA>,<NA>,intel hd graphics 620,NaN,NaN,"$1,999.00"
973,dell,latitude e6410,14 inches,silver,128 gb,core i3-520m,<NA>,<NA>,<NA>,8 gb,windows 7 professional,NaN,integrated,<NA>,<NA>,NaN,2.4 ghz,3.4,$111.05
981,dell,latitude e7440,14 inches,NaN,256 gb,core i5,<NA>,<NA>,<NA>,8 gb,windows 10,backlit keyboard,integrated,<NA>,<NA>,intel hd graphics,NaN,2.7,$133.55
1487,dell,latitude,13.3 inches,silver,128 gb,core i5-2520m,<NA>,<NA>,<NA>,8 gb,windows 7 professional,NaN,integrated,<NA>,<NA>,intel hd graphics 3000,NaN,3.5,$289.95
1804,panasonic,toughbook cf 20 mk2,10.1 inches,NaN,256 gb,intel core i5,<NA>,<NA>,<NA>,8 gb,windows 10 pro,support stylus,integrated,<NA>,<NA>,intel hd graphics 615,NaN,NaN,$508.74
1864,panasonic,toughbook cf 31,13 inches,NaN,500 gb,core i5-655k,<NA>,<NA>,<NA>,16 gb,windows 10 home,NaN,integrated,<NA>,<NA>,intel hd graphics,2.3 ghz,5.0,$595.97
2056,panasonic,toughbook cf 53 mk4,14 inches,silver,1 tb,core i5,<NA>,<NA>,<NA>,16 gb,windows 10 pro,fingerprint,integrated,<NA>,<NA>,NaN,NaN,NaN,$737.99
2073,panasonic,toughbook cf 54 mk1,14 inches,grey,512 gb,intel core i5,<NA>,<NA>,<NA>,8 gb,windows 10 pro,anti glare coating,integrated,<NA>,<NA>,NaN,NaN,3.3,$749.89


Additionally, there are also many cases where the `brand` is in the `model` column. This is data duplication and serves no purpose to the data.

In [810]:
# Reasoning: I remove all instances of the brand occurring in the model column.
#            This is redundant data and serves no purpose
def removeBrandFromModel(row):
    if not pd.isna(row['model']) and not pd.isna(row['brand']):
        if row['brand'] in str(row['model']):
            row['model'] = row['model'].replace(row['brand'], '')
    return row
df = df.apply(removeBrandFromModel, axis=1)

## The `color` column

Before we begin with standardising this column, we need to take care of a few non-standard values

In [811]:
problematic_colors = {
    'rgb backlit': {
        'special_features': 'rgb backlit keyboard'
    },
    'touchscreen': {
        'special_features': 'touchscreen'
    },
    'evo i7-1260p': {
        'cpu': 'evo i7-1260p'
    }
}
manualClean(df, problematic_colors, 'color', True)

Next we begin with standardising the colours of the laptops. 

Lets have a look at how many colours there currently are in the dataframe:

In [812]:
df['color'].unique()

array(['silver', 'black', 'grey, gray, platinum silver', 'gary', 'gray',
       nan, 'gray, platinum silver', 'carbon gray', 'titan gray',
       'pure silver', 'platinum silver', 'eclipse gray', 'dark moss',
       'natural silver', 'graphite black', 'graphite', 'obsidian black',
       'aluminum', 'arctic grey', 'platinum', 'blue', 'grey', 'green',
       'slate grey', 'silver/carbon fiber', 'sky', 'shadow grey',
       'burgundy red', 'midnight', 'mecha gray', 'off black', 'gold',
       'mineral grey', 'mist blue', 'punk pink', 'shale black',
       'star black', 'carbon fiber', 'gray, silver', 'shadow black',
       'white', 'fog blue', 'space gray', <NA>, 'cloud grey',
       'phantom grey', 'dark side of the moon', 'basalt grey',
       'matte black', 'mystic blue', 'indie black', 'stormy grey',
       'mystic black', 'phantom grey with speckles', 'ink black',
       'galaxy black', 'titan gray dull', 'aluminum titan gray',
       'dark shadow grey', 'royal blue', 'jade black', 

Thats way too many colours, some of there are also very similar.

To standardise them, we define a dictionary containing regex that match certain colours and then replace them with the correct colour.

In [813]:
# Reasoning: Reduces variability in the dataset due to misspellings, variations and interpretations
#            Reducuing the variability also helps with categorization which is use for data exploration and visualization
#            A part of this process also ensures contextual standardization which is crucial where colours are named uniquely 
#            for marketing or specific contexts (e.g. thunder black or platinum titan)
#            Need to first handle the cases that contain multiple conflicting standard colours
colorHashMap = {
    r".*(information not available|acronym).*": pd.NA,             #TODO Need to justify these
    r'titanium blue-black-dark blue-black|black/white': 'black',    #
    r'cover: red ; inner/keyboard: black': 'red',                   #
    r'ice blue \+ iron grey': 'grey',                               #TODO
    r".*(silver|sliver|aluminum|platinum|light titan|platinum titan).*": "silver",
    r".*(black|dark side of the moon|thunder balck|carbon fiber).*": "black",
    r".*(white).*": "white",
    r".*(grey|gray|gary|graphite|mercury|dark ash|dark metallic moon).*": "grey",
    r".*(red).*": "red",
    r".*(blue|cobalt|sky|dark teal|apollo|midnight).*": "blue",
    r".*(green|sage|soft mint|dark moss).*": "green",
    r".*(punk pink|electro punk|rose gold).*": "pink",
    r".*(gold).*": "gold",
    r".*(almond|beige mousse|lunar light|dune).*": "beige"          #TODO Need to justify this
}
df['color'] = df['color'].replace(colorHashMap, regex=True)

How many colours are there now?

In [814]:
df['color'].unique()

array(['silver', 'black', 'grey', nan, 'green', 'blue', 'red', 'gold',
       'pink', 'white', <NA>, 'beige'], dtype=object)

## The `screen_size` and `price` columns

These two columns store very similar data and the data is entirely clean. There are no bad values and hence the extraction of the values is quite simple

In [815]:
def extract_numerical_value(price):
    if pd.isna(price):
        return np.NaN
    numerical_value = re.search(r'(?P<value>\d+(?:\.\d+)?)', price.replace(",", "")).groupdict().get('value')
    return float(numerical_value)

# Remove 'inches' from screen sizes and '$' from prices. Convert these values to floats
# Reasoning: Removing 'inches' and '$' allows the values to be converted from a categorical string type to a continuous scale.
#            This allows for better data visualisation.
df['screen_size'] = df['screen_size'].apply(extract_numerical_value)
df['price'] = df['price'].apply(extract_numerical_value)

## The `ram` and `harddisk` column

These two columns are both in the same unit i.e. Bytes.
This makes it convenient to handle the standardisation of both of them in one go.

In [816]:
print(df['ram'].unique())
print()
print(df['harddisk'].unique())

['16 gb' '64 gb' '32 gb' '8 gb' nan '12 gb' '4 gb' '24 gb' '40 gb' '5 gb'
 '1 gb' '48 gb' '2 gb' '31.99 gb' '6 gb' '20 gb' '128 gb' '64 mb']

['256 gb' '2 tb' '1000 gb' '2000 gb' nan '512 gb' '128 gb' '4000 gb'
 '1 tb' '2048 gb' '16 gb' '1024 gb' '4 tb' '6000 gb' '3 tb' 512 '8000 gb'
 '1256 gb' 256 '320 gb' '32 gb' '64 gb' '160 gb' '120 gb' '500 gb' '8 tb'
 '64 mb' 64 '32 mb' '250 gb' '1.5 tb' '1512 gb' '128 mb' '65 gb' '8 gb'
 '240 gb' '480 gb' '512 mb' '256gb' '300 gb']


There is however a slight difference - It is extremely unlikely for a laptop to have over a terabyte of ram but possible to have a terabyte of harddisk. This is evident in the dataset.

A good way to handle this is by having a single function which handles converting and extracting the numerical value. Lets call it `convert_to_gb`

In [817]:
def convert_to_gb(size, tb_possible = False):
    # TODO check the correctness of this function, especially with NaNs
    if pd.isna(size): # Check for NaN
        return np.NaN

    if not isinstance(size, str):
        return round(float(size))

    if size.endswith('tb'):
        return round(float(size.replace('tb', '')) * 1000)
    
    if size.endswith('gb'):
        return round(float(size.replace('gb', '')))
    
    elif size.endswith('mb'):
        return round(float(size.replace('mb', '')))
    else:
        if tb_possible and float(size) < 16:
            return float(size) * 1000
        else: 
            return float(size)

Now that we have a function that can smoothly convert all the values into gigabytes we can just apply it to every value in each column.

In [818]:
# Adjust harddisk sizes to be more consistent
df['harddisk'] = df['harddisk'].apply(convert_to_gb, tb_possible=True)
df['ram'] = df['ram'].apply(convert_to_gb, tb_possible=False)

## The `OS` column

First lets have a look at what sort of data is in this column

In [819]:
df['OS'].unique()

array(['windows 10 pro', 'windows 11 home', 'windows 10 home',
       'windows 11 pro', 'windows 10', 'windows 11', 'mac os',
       'windows 7', nan, 'chrome os', 'windows pro', 'macos 10.12 sierra',
       'macos 12 monterey', 'windows 10 home 64',
       'windows 10 pro (english/french/spanish)',
       'windows 10 pro (64-bit)', 'win 10 pro / free upgrade to win 11',
       'microsoft windows 10 professional (64bit)', 'no',
       'windows 7 professional', 'windows 10 s', 'windows 8',
       'windows 11 s', 'windows', 'linux',
       'windows 10 pro 64 bit multi-language support english/french/spanish',
       'windows 10 pro 64 bit-multi-language support english/french/spanish.',
       'windows 10 pro 64-bit', 'unknown', 'windows 11 home s',
       'windows 10 64 bit-multi-language supports english/spanish/french',
       'windows 8.1',
       'win 10 pro 64 bit multi-language support english/french/spanish(renewed)',
       'win 10 pro 64 bit multi-language support english/frenc

Similarly to how we handled colours, we can create a regex hashmap to map all the current values to a more standardised system.

In [820]:
# Convert os systems to standard systems
# Reasoning: Removes unnecessary details and simplifies the OSs to simplistic values
osHashMap = {
    r".*(10 pro).*": "windows 10 pro",
    r".*(11 pro).*": "windows 11 pro",
    r".*(windows 10|win 10).*": "windows 10 home",
    r".*(windows 11 home|win 11 multi-home).*": "windows 11 home",
    r'.*(windows 11 s).*': 'windows 11',
    r".*(windows 7 professional).*": "windows 7 pro",
    r'.*(windows 7 home).*': 'windows 7',
    r'.*(windows pro).*': 'windows',
    r'.*(macos 10.12 sierra).*': 'mac os sierra',
    r'.*(macos 12 monterey).*': 'mac os monterey',
    r'.*(unknown|no).*': pd.NA
}
df['OS'] = df['OS'].replace(osHashMap, regex=True)

#TODO Dont just delete, keep it, just delete the column value
df = df[df['OS'] != 'hp thinpro']

Now lets see what the values look like:

In [821]:
df['OS'].unique()

array(['windows 10 home', 'windows 11 home', 'windows 11 pro',
       'windows 11', 'mac os', 'windows 7', nan, 'chrome os', 'windows',
       'mac os sierra', 'mac os monterey', <NA>, 'windows 7 pro',
       'windows 8', 'linux', 'windows 8.1', 'windows 8 pro',
       'windows 10 pro'], dtype=object)

## The `cpu` column

First let's have a look at what sort of data is in this column

In [822]:
df['cpu'].unique()

array(['core i5 6200u', 'core i7 family', 'core i7', 'core i5',
       'amd ryzen 7', 'ryzen 5', 'unknown', 'amd r series',
       'core i5 family', 'intel core i5', 'intel core i5 1145g7',
       'intel core i7', 'intel core i5-1135g7', nan, 'ryzen 5 4600h',
       'ryzen 7', 'core i7-10700t', 'core i7 12700h', 'core i3',
       'ryzen 7 5800h', 'intel mobile cpu', 68000, 'apple m1',
       '1.2ghz cortex a8 processor', 'amd ryzen 5 5500u', 'corei7-10750h',
       'intel core i7-1165g7', 'apple m2', 'intel core i7 1185g7', 8032,
       'core i7 7500u', 'core i5-1035g1', 'amd ryzen 7 5800h',
       'evo i7-1260p', 'ryzen 7 2700u', 'ryzen 9', 'core i9',
       'core i9 7960x', 'snapdragon', 'intel core i7 extreme',
       'core i7-8650u', 'core i5 7200u', 'intel core i9',
       'core i7 11800h', 'intel core duo', 'core i7 8750h',
       'intel core i7 12800h', 'core i7 8550u', 'amd a series',
       'amd ryzen 9 5900x', 'core i7-1065g7',
       'intel core m-5y10 processor', 'a-series 

One thing that is immediately obvious is that some rows contain the cpu speed of the given processor. This can be extracted into the `cpu_speed` column

In [823]:
df['cpu'] = df['cpu'].str.replace(r'_|-', ' ', regex=True)

def extractCPUspeed(row):
    pattern = r'(?: ?(?P<cpu_speed>\d\.\d) ?ghz ?)'
    if match := re.search(pattern, str(row['cpu'])):
        if speed := match.groupdict().get('cpu_speed'):
            row['cpu_speed'] = speed
            row['cpu'] = re.sub(pattern, '', str(row['cpu']))
    return row
df = df.apply(extractCPUspeed, axis=1)

Lets look at the column again

In [824]:
df['cpu'].unique()

array(['core i5 6200u', 'core i7 family', 'core i7', 'core i5',
       'amd ryzen 7', 'ryzen 5', 'unknown', 'amd r series',
       'core i5 family', 'intel core i5', 'intel core i5 1145g7',
       'intel core i7', 'intel core i5 1135g7', nan, 'ryzen 5 4600h',
       'ryzen 7', 'core i7 10700t', 'core i7 12700h', 'core i3',
       'ryzen 7 5800h', 'intel mobile cpu', 'apple m1',
       'cortex a8 processor', 'amd ryzen 5 5500u', 'corei7 10750h',
       'intel core i7 1165g7', 'apple m2', 'intel core i7 1185g7',
       'core i7 7500u', 'core i5 1035g1', 'amd ryzen 7 5800h',
       'evo i7 1260p', 'ryzen 7 2700u', 'ryzen 9', 'core i9',
       'core i9 7960x', 'snapdragon', 'intel core i7 extreme',
       'core i7 8650u', 'core i5 7200u', 'intel core i9',
       'core i7 11800h', 'intel core duo', 'core i7 8750h',
       'intel core i7 12800h', 'core i7 8550u', 'amd a series',
       'amd ryzen 9 5900x', 'core i7 1065g7',
       'intel core m 5y10 processor', 'a series dual core a4 3300m',

Based on the structure and format of the values, we can construct a good way to split the values into 3 component columns:
- `cpu_brand`
- `cpu_series`
- `cpu_model`

We can use a similar regex hashmap structure as before but this time we can sub-divide the hashmap based on the brand. This will be useful for imputing the brand when it isnt specified in the cell.

In [825]:
brandPatterns = {
    'intel': {
        r'^(?:intel )?core (?P<cpu_series>i[3579]( extreme)?)(?: family)?$',
        r'^(?:intel )?core\s?(?P<cpu_series>i[3579])(?:[ -])(?P<cpu_model>(?:\d{3,5}(?:[uthxyqmk]{1,2}))|(?:\d{4}g\d))$',
        r'^(?:intel )?(?:(?P<cpu_series>celeron|pentium)\s?)(?P<cpu_model>[np]\d{4}|\d{4}u|n|d|4)?$',
        r'(?:intel )?(?P<cpu_series>core m\d?)(?: (?P<cpu_model>8100y|5y10))?',
        r'^(?:intel )?(?:(?P<cpu_series>mobile) cpu)$',
        r'^(?:intel )?(?P<cpu_series>atom|xeon)$',
        r'^(?P<cpu_series>8032)$',
        r'^(?:intel )?(?P<cpu_series>core (?:2 duo|2|duo))(?: quad)?(?: (?P<cpu_model>p\d{4}))?$',
        r'^(?P<cpu_series>evo i7) (?P<cpu_model>1260p)$',
        r'^(?P<cpu_series>atom) (?P<cpu_model>z8350)$'
    },
    'amd': {
        r'^(?:amd )?(?P<cpu_series>ryzen\s?[3579])(?:\s(?P<cpu_model>\d{4}[hux]))?$',
        r'(?:amd )?(?:kabini )?(?P<cpu_series>[ar] series|a\d{1,2})(?: (?P<cpu_model>\d{4}[km]))?',
        r'^(?P<cpu_series>athlon(?: silver)?)(?:\s(?P<cpu_model>\d{4}u))?$',
        r'(?P<cpu_series>cortex)(?: (?P<cpu_model>a\d{1,2}))?',
        r'^a series dual core (?P<cpu_series>a4) (?P<cpu_model>3300m)$'
    },
    'mediatek': {
        r'^(?:mediatek)[ _](?P<cpu_model>.*)$',
    },
    'apple': {
        r'^(?:apple )(?P<cpu_model>m[12])?$',
    },
    'arm': {
        r'^arm (?P<cpu_series>7100)$'
    },
    'snapdragon': {
        r'^snapdragon$',
    },
    'motorola': {
        r'^(?P<cpu_series>68000)$',
    },
    pd.NA: {
        r'^unknown|others$',
    }
}

Now that we have the hashmap which encodes all the patterns. We can use a function to apply all the regexes to each row of the column.

In [826]:
def extractCPUdetails(row):
    for brand, patterns in brandPatterns.items():
            for pattern in patterns:
                if match := re.search(pattern, str(row['cpu'])):
                    row['cpu_brand'] = brand

                    if match.groupdict().get('cpu_series') and pd.isna(row['cpu_series']):
                        row['cpu_series'] = match.group('cpu_series')

                    if match.groupdict().get('cpu_model') and pd.isna(row['cpu_model']):
                        row['cpu_model'] = match.group('cpu_model')         
    return row

df = df.apply(extractCPUdetails, axis=1)

## The `special_features` column

Similarly to before, we construct a regex dictionary which stores all the mappings of the special features. 

In [827]:
patterns = {
    r'(anti-? ?(?:gla(?:re)?)|reflection)': 'anti-glare',
    r'fingerprint': 'fingerprint sensor',
    r'bezel|(?:infinity|nano)edge|narrow': 'thin bezel',
    r'backli(?:gh)?t': 'backlit keyboard',
    r'^(?:stylus )?pen$|active stylus': 'stylus',
    r'(?:high definition|hd) audio': 'hd audio',
    r'support stylus': 'stylus support',
    r'dolby': 'dolby audio',
    r'spill[ -]resistant': 'spill-resistant',
    r'stereo|speakers': 'stereo speakers',
    r'multi[ -]touch': 'multi-touch',
    r'chiclet': 'chiclet keyboard',
    r'alexa': 'amazon alexa',
    r'corning gorilla glass': 'corning gorilla glass',
    r'water proof|water resistant': 'water-resistant',
    r'lightweight|light and compact design': 'lightweight',
    r'killer wifi 6e': 'wifi',
    r'touch ?screen': 'touchscreen',
    r'anti-ghost key': 'anti-ghost keys',
    r'keypad': 'numeric keypad',
    r'bluetooth|memory card slot|2-in-1|trackpoint|anti-smudge|miracast|microphone|security slot|camera|detachable|144hz refresh rate|ergonomic': '_KEEP_'
}

Now, lets create a function which will iterate through all the rows, converting all the special features into the standardised values.

Additionally, lets replace all occurrences of 'wifi & bluetooth' with 'wifi,bluetooth'.

In [828]:
df['special_features'] = df['special_features'].replace(r'wifi & bluetooth', 'wifi,bluetooth', regex=True)

def enumSpecialFeatures(row):
    if pd.isna(row['special_features']):
        return row

    #Split original rows based on commas and remove all leading and trailing whitespace
    features_split = str(row['special_features']).split(',')
    stripped_features = [feature.strip() for feature in features_split]

    # Apply each regex to each feature listed in the row
    for idx, feature in enumerate(stripped_features):
        for pattern, mapped_value in patterns.items():
            if re.search(pattern, feature):
                stripped_features[idx] = mapped_value if mapped_value != '_KEEP_' else feature
                break
        # Else is executed if the break statement was never reached.
        else:
            stripped_features[idx] = pd.NA

    # Remove all empty string and NaN from the special features
    stripped_features = [feature for feature in stripped_features if (not pd.isna(feature) and feature != '')]

    # Turn into set to remove duplicates, convert back to list, sort the list
    # Reasoning: Sorting the list will be helpful later when checking for duplicate 
    #            laptops that just had their special features in different orders.
    stripped_features = sorted(list(set(stripped_features)))

    # Convert the list of features back into comma seperated strings
    # Lists containing no features are mapped to NaN
    row['special_features'] = ",".join(stripped_features) if len(stripped_features) > 0 else pd.NA

    return row

df = df.apply(enumSpecialFeatures, axis=1)

## The `cpu_speed` column

Before beginning standardisation, lets examine the current data:

In [829]:
df['cpu_speed'].unique()

array([nan, '3.6 ghz', '2.1 ghz', '4.4 ghz', '3 ghz', '1.7 ghz',
       '3.5 ghz', 1.9, '3.7 ghz', '2133 mhz', '4.2 ghz', '2.6 ghz',
       '1.8 ghz', '1.2', '3.4 ghz', '4.7 ghz', 3.4, '1.9 ghz', '4.6 ghz',
       '2.2 ghz', '4.5 ghz', 4.7, '1.6 ghz', '3.2 ghz', '3200 hz',
       '2.4 ghz', '2.5 ghz', 3.3, '5.2 ghz', '5 ghz', '3.49 ghz',
       '2.9 ghz', '2.8 ghz', '4.8 ghz', '0.9 ghz', '5.6 ghz', '2.3 ghz',
       '2 ghz', '5.1 ghz', '1.3 ghz', '2.7 ghz', '2.66 ghz', '1.1 ghz',
       '1 ghz', '1.2 ghz', '3.9 ghz', '3.8 ghz', '1.4 ghz', 2.6, '1.5',
       '4 ghz', 2.4, '4.3 ghz', 1.8, '3.3 ghz', '2.53 ghz', '1.5 ghz',
       '2.3'], dtype=object)

It is clear that the values are all in a fairly standard format. Since there are currently no processors operating faster than 10Ghz, it is fair to say any speed above 10 is written in MHz and should be converted to GHz

In [830]:
# Adjust CPU speeds to be more consistent with the same units
def standardiseCPU(cpu_speed):
    match = re.search(r'((\d+\.\d+|\d+))', str(cpu_speed))
    if match:
        value = float(match.group())
        if value > 10:
            value /= 1000
        return value

df['cpu_speed'] = df['cpu_speed'].apply(standardiseCPU)


## The `graphics` column

Examining the data:

In [831]:
print(df['graphics'].unique())

print(df['graphics_coprocessor'].unique())

['integrated' 'iris xe graphics' 'radeon graphics' 'nvidia geforce mx250'
 'nvidia geforce rtx a500' 'dedicated' 'integrated, dedicated' nan
 'uhd graphics' 'rtx t600' 't550' 'rtx 3060' 'nvidia geforce mx350'
 'nvidia geforce rtx 3050' 'nvidia geforce rtx 3050 ti'
 'nvidia geforce rtx 2050' 'iris x graphics' 'rtx a3000' 'intel xe'
 'iris xe' 'nvidia quadro t550' 'nvidia geforce rtx 3060' 'quadro t500'
 'nvidia geforce rtx 4050' 'rtx a1000' 'intel iris xáμ‰'
 'nvidia geforce rtx 4060' 'radeon 680m graphics' 'rtx 4070' 'rtx 3050 ti'
 'nvidia geforce rtx 2070 super' 'gt2 graphics'
 'geforce rtx 3070 ti,iris xe graphics' 'nvidia rtx3070'
 'amd radeon rx 540' 'rtx2050' 'rtx a500' 'nvidia quadro t2000'
 'rtx 3070 ti' 'nvidia geforce rtx 3070 ti' 't500,iris xe graphics'
 'geforce rtx 3080' 'nvidia geforce rtx2060' 'rtx a2000'
 'nvidia geforce mx550' 'intel arc a370m' 'nvidia t1200'
 'nvidia rtx a3000' 'nvidia quadro rtx 3000' 't1200' 'nvidia rtx a2000'
 'nvidia geforce rtx 2070' 'amd radeon r

It is evident this data is very unstructured.
- The `graphics` should only contain integrated or dedicated however it contains other values
- The values in the `graphics_coprocessor` have a similar format but the data would be better off extracted into finer columns.

Ideally, for each laptop we should store
- `graphics` - Integrated or Dedicated graphics
- `graphics_brand` - The brand of the GPU
- `graphics_details` - The specific details of the gpu

This would allow the original data to remain intact whilst also allowing laptops to be compared based on specific sub-parameters.

Before we begin standardising, we need to address some specific values. These are mostly values that do not belong in the `graphics` columns and instead belong in the `cpu` columns.

In [832]:
problematic_gpus = {
        'amd athlon silver': {
            'cpu_model': 'silver',
        },
        'inter core i7-8650u': {
            'cpu_series': 'i7',
            'cpu_model': '8650u',
        },
        'amd athlon': {},
        'intel celeron': {  
            'cpu_brand': 'intel',
            'cpu_series': 'celeron',
        },
        'm1 pro': {
            'cpu_brand': 'apple',
            'cpu_series': pd.NA,
            'cpu_model': 'm1 pro',
        },
        'xps9300-7909slv-pus': {
            'model': 'xps9300-7909slv-pus',
        },
        'integrated intel iris xe plus gpu': {
            'graphics': 'dedicated',
        }
    }
manualClean(df, problematic_gpus, 'graphics_coprocessor', True)

Similarly to the previous methods, the way to approach this column is to use regex.
This time however, for each value, it would be ideal to impute missing `graphics` and `graphics_brand` values. 

This is done by storing each regex in a sort of directory. This means that whenever a regex matches, by traversing up the dictionary, it is clear what the brand and graphics are.

In [833]:
graphicsPatterns = {
    'dedicated': {
        'nvidia': {
            r'nvidia optimus graphics': 'optimus graphics',
            r'(?P<graphics_details>quadro rtx\s*\d{4}(?:\s*(?:ti|super))?)': '__EXTRACT__',
            r'(?P<graphics_details>rtx\s*\d{4}(?:\s*(?:ti|super))?)': '__EXTRACT__',
            r'(?P<graphics_details>rtx\s*a\d{1,2}00)': '__EXTRACT__',
            r'nvidia geforce (?P<graphics_details>mx\d{2}0)': '__EXTRACT__',
            r'(?P<graphics_details>(?:quadro )?t\d{3,4})': '__EXTRACT__',
            r'geforce(?: rtx| gtx)?(?: (?P<graphics_details>\d{4}(?:\s?(?:oc|ti))?|a3000|940mx))': '__EXTRACT__',
            r'(?P<graphics_details>quadro (?:p\d{3,4}|k?\d{4}m?))': '__EXTRACT__',
            r'^(?P<graphics_details>qn20-m1-r)$': '__EXTRACT__'
        },  
        'amd': {
            r'amd (?P<graphics_details>radeon r\d)': '__EXTRACT__',
            r'wx vega': 'radeon pro wx vega m gl',
            r'(?:amd )?(?P<graphics_details>(?:ati mobility )?radeon(?: graphics)?(?: (?:hd|pro|rx))? \d{3,4}(?!.*m))': '__EXTRACT__'
        },
        'intel': {
            r'intel (?P<graphics_details>arc a\d{3}m)': '__EXTRACT__'
        }
    },
    'integrated': {
        'intel': {
            r'(?P<graphics_details>uhd(?: graphics)?(?: premium| \d{3})?)': '__EXTRACT__',
            r'(?P<graphics_details>hd graphics(?: \d{3,4})?)(?!nvidia)': '__EXTRACT__',
            r'iris plus': 'iris xe plus graphics',
            r'iris(?!.*gpu)|intel xe': 'iris xe graphics',
            r'intel (?P<graphics_details>hd(?: \d{3,4})?)': '__EXTRACT__',
            r'(?P<graphics_details>gt2 graphics)': '__EXTRACT__',
            r'^(?P<graphics_details>hd integrated graphics)$': '__EXTRACT__',
            r'intel (?P<graphics_details>\d{3}u)': '__EXTRACT__'
        },
        'nvidia': {
            r'nvidia geforce gtx (?P<graphics_details>\d{3}m)': '__EXTRACT__'
        },
        'amd': {
            r'amd (?P<graphics_details>radeon (?:rx )?vega (?:3|8|9|11))': '__EXTRACT__',
            r'^(?:integrated )?(?:amd )?radeon(?: graphics)?$': 'radeon graphics',
            r'(?P<graphics_details>radeon (?:(?:rx|hd) )?\d{3,4}m)': '__EXTRACT__',
            r'^amd (?P<graphics_details>integrated graphics)$': '__EXTRACT__',
            r'^amd (?P<graphics_details>radeon graphics 5)$': '__EXTRACT__'
        },
        'mediatek': {
            r'mediatek': pd.NA
        },
        'imagination technologies': {
            r'^(?P<graphics_details>powervr gx6250)$': '__EXTRACT__'
        },
        'apple': {
            r'apple integrated graphics': pd.NA
        },
        'qualcomm': {
            r'(?P<graphics_details>adreno 618)': '__EXTRACT__'
        },
        'arm': {
            r'(?P<graphics_details>mali-g[57]2 (?:mp3|2ee mc2)?)': '__EXTRACT__'
        },
        pd.NA: {
            r'^embedded$|^integrated(?: ?graphics)?$|^shared$': pd.NA
        }
    },
    '__KEEP__': {
        'nvidia': {
            r'nvidia': pd.NA
        },
        'intel': {
            r'intel': pd.NA
        }
    },
    pd.NA: {
        pd.NA: {
            r'^$': pd.NA,
        }
    },
}

Next, lets define a function that will use this dictionary to extract the values into the finer columns

In [834]:
def extractGPUdetails(row):
    for graphics_type, graphics_brands in graphicsPatterns.items():
        for graphics_brand, patterns in graphics_brands.items():
            for pattern, overwriteValue in patterns.items():
                if match := re.search(pattern, str(row['graphics_coprocessor'])):
                    if graphics_type != '__KEEP__':
                        row['graphics'] = graphics_type
                    row['graphics_brand'] = graphics_brand

                    if (pd.isna(row['graphics_details']) or row['graphics_details'] == ''):
                        if details := match.groupdict().get('graphics_details'):
                            row['graphics_details'] = details
                        else:
                            row['graphics_details'] = overwriteValue
                    
    row['graphics_coprocessor'] = pd.NA
    return row

Now that we have this function we can define the function that will wrap all this function together.

It will
- Fix some spelling errors
- Trim each value to remove extra whitespace
- Move all values that are not integrated or dedicated into the `graphics_coprocessor` column
- Remove all non-alphanumeric characters
- Execute `extractGPUDetails()` on the `graphics_coprocessor` column


In [835]:
# Fix spellings and duplication
columns = ['graphics', 'graphics_coprocessor']
replacements = {r'integreted|intergrated': 'integrated', 
                r'integrated, dedicated': 'dedicated',
                r'geforcer': 'geforce',
                r'grpahics|\bgraphic\b': 'graphics',
                r'\bx\b': 'xe',
                r'rtx(?! )': 'rtx ',
                r'\bgt\b': 'gtx'}
df[columns] = df[columns].replace(replacements, regex=True)

# Reasoning: Remove unnecessary whitespace from the value
df[columns] = df[columns].replace('\s+', ' ', regex=True)
for column in columns:
    df[column] = df[column].str.strip(' ')

# Move gpu info to the other column
def moveGPUinfo(row):
    if pd.notna(row['graphics']):
        if row['graphics'] not in ['integrated', 'dedicated']:
            row['graphics_coprocessor'] = row['graphics']
            row['graphics'] = pd.NA
    return row
df = df.apply(moveGPUinfo, axis=1)

# Remove all non-alphanumeric characters
df['graphics_coprocessor'] = df['graphics_coprocessor'].replace(r'[^A-Za-z0-9, \-]', '', regex=True)

df = df.apply(extractGPUdetails, axis=1)

## Rename columns

In [836]:
columnNameHashMap = {
    'screen_size': 'screen_size_inches',
    'color': 'colour',
    'harddisk': 'harddisk_gb',
    'ram': 'ram_gb',
    'cpu_speed': 'cpu_speed_ghz',
    'price': 'price_usd',
    'OS': 'os'
}
df = df.rename(columns=columnNameHashMap)

## Post Processing

In [837]:
categorical_columns = [
    'brand', 
    'model', 
    'colour', 
    'cpu_brand', 
    'cpu_series', 
    'cpu_model', 
    'os', 
    'special_features', 
    'graphics', 
    'graphics_brand', 
    'graphics_details'
]
integer_columns = [
    'harddisk_gb', 
    'ram_gb'
]
float_columns = [
    'screen_size_inches', 
    'cpu_speed_ghz', 
    'rating',
    'price_usd'        
]


# Drop old columns
df = df.drop(columns=['cpu', 'graphics_coprocessor'])

# Update the types for all the columns and fill missing values with flag value
df[categorical_columns] = df[categorical_columns].fillna('_MISSING_')
df[categorical_columns] = df[categorical_columns].astype(str)
df[integer_columns] = df[integer_columns].astype('Int64')
df[float_columns] = df[float_columns].astype(float)

# Clean string data
df[categorical_columns] = df[categorical_columns].replace('\s+', ' ', regex=True)
df[categorical_columns] = df[categorical_columns].apply(lambda x: x.str.strip(', '))

df = df[df[''] != '']

df = df.drop_duplicates(ignore_index=True)
df.reset_index(drop=True, inplace=True)

## Output the cleaned data

In [838]:
outputDataToFile(df)